# **I &nbsp;&nbsp;&nbsp; Import Libraries**

In [1]:
from ipycanvas import Canvas
from IPython.display import display, HTML
from PIL import Image

import ipywidgets as widgets
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

import io

from Utils.feature_vector import FeatureVector

# **II &nbsp;&nbsp;&nbsp; Import Model**

In [2]:
with open("../Data/Output/edge_based.pkl", "rb") as f:
    model = pickle.load(f)

# **III &nbsp;&nbsp;&nbsp; Design UI**

## **1 &nbsp;&nbsp;&nbsp; Right Side**

### **2.1 &nbsp;&nbsp;&nbsp; Prediction Box**

In [3]:
result_lbl = widgets.Label(
    "RESULT",
    style={"font_size": "20px"}
)
prediction_lbl = widgets.Label(
    "Prediction:", 
    style={
        "font_weight": "bold",
        "font_size": "30px"
    }
)
label_lbl = widgets.Label(
    "?",
    layout={
        "height": "70px",
        "align_items": "center"
    },
    style={
        "font_weight": "bold",
        "font_size": "70px"
    }
)

lbl_area = widgets.HBox(
    [ widgets.VBox([ result_lbl, prediction_lbl ], layout={ "width": "250px" }), label_lbl ],
    layout={
        "width": "350px",
        "height": "100px",
        "border": "2px solid #e0f7e9"
    }
)

In [4]:
def update_label_area(value):
    global label_lbl
    label_lbl.value = f"{value}"

In [5]:
HTML(
    """
    <style>
    .rounded-label {  
        border-radius: 12px;        
        overflow: hidden;
        display: start;
        padding: 0px;
        font-family: Comic Sans MS, cursive;
        background-color: #e0f7e9;
        justify-content: space-around;
        align-items:center
    }
    </style>
    """
)

In [6]:
lbl_area.add_class("rounded-label")
display(lbl_area)

### **2.2 &nbsp;&nbsp;&nbsp; Plot Box**

In [7]:
plot_out = widgets.Output(layout={
    "width": "350px",
    "height": "450px",
    "margin": "10px 0px",
    "overflow": "visible"
})

In [8]:
def plot_out_probability(probs):
    global plot_out
    with plot_out:
        plot_out.clear_output()
        
        plt.figure(figsize=(8, 6))
        plt.bar(x=range(len(probs)), height=probs, width=0.5, color="#FFC073", linewidth=0)
        
        # Ticks the x, y axis
        plt.xticks(ticks=np.arange(10), labels=range(10))
        plt.yticks([])
        
        # Remove borders
        ax = plt.gca()
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.tick_params(bottom=False, left=False)
        
        plt.tight_layout()
        plt.show()

### **2.3 &nbsp;&nbsp;&nbsp; Merge UI**

In [9]:
right_side = widgets.VBox(
    [ lbl_area, plot_out ],
    layout={
        "width": "400px",
        "height": "500px",
        "margin": "10px",
        "align_items": "center"
    }
)

In [10]:
right_side.add_class("rounded-box")
display(right_side)

## **2 &nbsp;&nbsp;&nbsp; Left Side**

### **2.1 &nbsp;&nbsp;&nbsp; Drawing Canvas**

In [11]:
canvas = Canvas(
    width=300, height=300, 
    layout={ "width": "300px", "height": "300px" }, 
    sync_image_data=True
)

In [12]:
# Define drawing line
canvas.fill_style = 'black'
canvas.fill_rect(0, 0, canvas.width, canvas.height)

canvas.line_width = 20
canvas.stroke_style = "white"
canvas.line_cap = "round"    
canvas.line_join = "round"

In [13]:
# Event listeners
is_drawing = False
last_x, last_y = None, None

def on_mouse_down(x, y):
    global is_drawing, last_x, last_y
    is_drawing = True
    last_x, last_y = x, y

def on_mouse_move(x, y):
    global is_drawing, last_x, last_y
    if is_drawing:
        canvas.stroke_line(last_x, last_y, x, y)
        last_x, last_y = x, y

def on_mouse_up(x, y):
    global is_drawing
    is_drawing = False

canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

In [14]:
box = widgets.Box(
    [ canvas ],
    layout={
        "width": "300px",
        "height": "300px",
        "align_items": "center",
    }
)

In [15]:
HTML(
    """
    <style>
    .rounded-canvas {
        border-radius: 18px;        
        overflow: hidden;
        display: block;
        padding: 0px;
    }
    </style>
    """
)

In [16]:
box.add_class("rounded-canvas")
display(box)

Box(children=(Canvas(height=300, image_data=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x01,\x0…

### **2.2 &nbsp;&nbsp;&nbsp; Clear Button**

In [17]:
HTML(
    """
    <style>
    .rounded-button {  
        border-radius: 12px;        
        overflow: hidden;
        display: inline-block;
        padding: 0px;
        cursor: pointer;
        box-shadow: 0px 5px 5px 0px #999;
        transition: all 0.1s ease-in-out;
        font-family: Comic Sans MS, cursive;
    }
    .rounded-button:active {
        box-shadow: 0 2px #666;
        transform: translateY(3px);
    }
    .rounded-button:hover {
        filter: brightness(1.1);
    }
    </style>
    """
)

In [18]:
clear_btn = widgets.Button(
    description="Clear",
    layout={
        "width": "100px",
        "height": "40px",
        "border": "2px solid #E0E0E0"
    },
    style={
        "button_color": "#E0E0E0",
        "font_weight": "bold",
        "font_size": "20px",
    }
)

In [19]:
def clear_canvas(b):
    canvas.clear()
    canvas.fill_style = "black"
    canvas.fill_rect(0, 0, canvas.width, canvas.height)

clear_btn.on_click(clear_canvas)

In [20]:
clear_btn.add_class("rounded-button")
display(clear_btn)

Button(description='Clear', layout=Layout(border_bottom='2px solid #E0E0E0', border_left='2px solid #E0E0E0', …

### **2.3 &nbsp;&nbsp;&nbsp; Upload Button**

In [21]:
upload_btn = widgets.FileUpload(
    description="Upload Image",
    layout={
        "width": "200px",
        "height": "40px",
        "border": "2px solid #A3D5FF"
    },
    style={
        "button_color": "#A3D5FF",
        "font_weight": "bold",
        "font_size": "20px",
    },
    accept="image/*", 
    multiple=False
)

In [22]:
def handle_upload(change):
    if upload_btn.value:
        for file_info in upload_btn.value:
            img_bytes = file_info["content"]
            img = Image.open(io.BytesIO(img_bytes)).convert("L")
            img_array = np.asarray(img.resize((300, 300)))
            
            canvas.put_image_data(img_array)

upload_btn.observe(handle_upload, names="value")

In [23]:
upload_btn.add_class("rounded-button")
display(upload_btn)

FileUpload(value=(), accept='image/*', description='Upload Image', layout=Layout(border_bottom='2px solid #A3D…

### **2.4 &nbsp;&nbsp;&nbsp; Predict Button**

In [24]:
predict_btn = widgets.Button(
    description="Predict Label",
    layout={
        "width": "300px",
        "height": "40px",
        "border": "2px solid #FFC073"
    },
    style={
        "button_color": "#FFC073",
        "font_weight": "bold",
        "font_size": "30px"
    }
)

In [25]:
def get_mnist_image():
    data = canvas.get_image_data()[:, :, :3]
    img = Image.fromarray(data.astype(np.uint8))
    img = img.resize((28, 28))
    img_gray = img.convert("L")
    return np.array([img_gray])

def on_predict(b):
    img = get_mnist_image().reshape(1, 28, 28)
    feature_vector = FeatureVector()
    X = feature_vector.sobel_edge(img)

    probs = model.predict(X)[0]
    pred_label = np.argmax(probs)
    
    update_label_area(pred_label)
    plot_out_probability(probs)

predict_btn.on_click(on_predict)

In [26]:
predict_btn.add_class("rounded-button")
display(predict_btn)

Button(description='Predict Label', layout=Layout(border_bottom='2px solid #FFC073', border_left='2px solid #F…

In [27]:
btn_area = widgets.VBox(
    [ widgets.HBox([ clear_btn, upload_btn ], layout={ "height": "50px" }), predict_btn ],
    layout={
        "width": "400px",
        "height": "120px",
        "align_items": "center",
        "padding": "10px 0px"
    }
)
display(btn_area)

### **2.5 &nbsp;&nbsp;&nbsp; Merge UI**

In [28]:
HTML(
    """
    <style>
    .rounded-box {
        border: 1px solid #FFFFFF;  
        border-radius: 18px;        
        padding: 45px 0px 0px 0px;
        box-shadow: 0px 5px 5px 5px #999;
    }
    </style>
    """
)

In [29]:
left_side = widgets.VBox(
    [ box, btn_area ], 
    layout={
        "width": "401px",
        "height": "500px",
        "margin": "10px",
        "align_items": "center"
    }
)

In [30]:
left_side.add_class("rounded-box")

## **3 &nbsp;&nbsp;&nbsp; Merge UI**

In [31]:
HTML(
    """
    <style>
    .rounded-ui {
        border: 1px solid #FFFFFF;  
        border-radius: 18px;        
        padding: 10px 0px;
        margin: 10px 0px;
        box-shadow: 0px 5px 5px 5px #999;
    }
    </style>
    """
)

In [32]:
title = widgets.Label(
    "Predict Handwritten Digit Application",
    layout={
        "height": "70px",
        "align_items": "center"
    },
    style={
        "font_weight": "bold",
        "font_size": "50px",
        "font_family": "Comic Sans MS, cursive",
    }
)

main_components = widgets.HBox(
    [ left_side, right_side ],
    layout={
        "width": "900px",
        "height": "550px",
        "align_items": "center",
        "justify_content": "space-around",
    },
)

ui = widgets.VBox(
    [ title, main_components ],
    layout={
        "width": "1000px",
        "align_items": "center"
    }
)
ui.add_class("rounded-ui")
display(ui)